# Counting Inversions

The number of *inversions* in a disordered list is the number of pairs of elements that are inverted (out of order) in the list.  

Here are some examples: 
  - [0,1] has 0 inversions
  - [2,1] has 1 inversion (2,1)
  - [3, 1, 2, 4] has 2 inversions (3, 2), (3, 1)
  - [7, 5, 3, 1] has 6 inversions (7, 5), (3, 1), (5, 1), (7, 1), (5, 3), (7, 3)
  
The number of inversions can also be thought of in the following manner. 

>Given an array `arr[0 ... n-1]` of `n` distinct positive integers, for indices `i and j`, if `i < j` and `arr[i] > arr[j]` then the pair `(i, j)` is called an inversion of `arr`.

## Problem statement

Write a function, `count_inversions`, that takes an array (or Python list) as input, and returns a count of the total number of inversions present in the input.

Mergesort provides an efficient way to solve this problem.

In [65]:
def merge(lar,rar):
    
    
    merged = []
    lidx = 0
    ridx = 0
    
    inver_count = 0

    
    while lidx < len(lar) and ridx < len(rar):
        
        if lar[lidx] > rar[ridx]:
            merged.append(rar[ridx])
            ridx+=1
            
            # 剩下的元素要包含自己 
            larry_reamin_ele = (len(lar)-lidx )
            
            inver_count = inver_count + larry_reamin_ele
            print('inver_count:',inver_count)

            
        else:
            merged.append(lar[lidx])
            lidx+=1            


        
            
            
    if len(lar[lidx:]) > 0:
        merged += lar[lidx:]
    
    if len(rar[ridx:]) > 0:
        merged += rar[ridx:]
        
        
    print('merged:',merged)

    return merged,inver_count

def mergesort(arr):
    
    #base case 
    if len(arr) ==1:
        return arr,0
    
    #divide or split
    #mid = (0+len(arr)-1)//2 #會停不下來, 應該不至於呀? 要查一下
    mid = len(arr)//2
    leftarr = arr[:mid]
    righarr = arr[mid:]
    
    print('leftarr:',leftarr)
    print('righarr:',righarr)

    leftrst,lcnt = mergesort(leftarr)
    righrst,rcnt = mergesort(righarr)
    
    
    low_lay_cnt = lcnt + rcnt
    
    #merge
    mst,this_lay_cnt = merge(leftrst,righrst)
    
    
    
    acum_cnt = this_lay_cnt + low_lay_cnt
    
    return mst,acum_cnt
    
    
    

In [57]:
def merge_global_idx(arr,lstart,lend,rstart,rend):
    
    inver_count = 0
    
    lidx = lstart
    ridx = rstart
    mer_idx = 0
    
    leftleng = (lend-lstart+1)
    righleng = (rend-rstart+1)
    merged_output_length = leftleng   +  righleng
    merged = [None for _ in range(merged_output_length) ]
    
    while mer_idx < merged_output_length:
        
        #左邊比較小, 正常, 直接放到output
        if arr[lidx] <= arr[ridx]:
            merged[mer_idx] = arr[lidx]
            lidx +=1
        else:
            
            
        #####################
        # KEY IDEA: 
        # 左邊是sorted 過的, 左idx element 比右邊elemA大, 表示左邊 idx+1~結束 都會比 elemA大
        #####################
        #左邊比較大, Inversison HIT, count 增加,但不是 +1
        # 是加上左邊arry 剩餘還沒被處理過的數量 (lend-當下 lidx + 1)
        
        #ex: [2,5] [1,3,4]
        # lidx = 0, ridx = 0, 2 > 1, 此時因為剛開始, inver_count = 0
        # 而 lend = 1, 因此 cur_larr_reamin_eles = 2
        # l 已經是sorted, 所以如果 2 會大於 1, 2右邊的5,一定也 > 1, 因此
        # inversion 數量是 看發生的當下, 包含自己和剩下右邊的element 數
            
            cur_larr_reamin_eles = (lend- lidx +1)
            inver_count = inver_count + cur_larr_reamin_eles
            
            #一般放到rest
            merged[mer_idx] = arr[ridx]#output 都是放比較小的ele
            ridx +=1       
            
            
        
        mer_idx +=1
        
        #如果其中一個人已經沒elemt 了
        #直接把另一個剩下的element 接在 output 後面
        #break
        
        if lidx > lend:#左邊已經耗盡
            for i in range(ridx, rend+1):
                merged[mer_idx] = arr[i]#output 都是放比較小的ele
                mer_idx +=1
            break
        
        if ridx > rend:
            for i in range(lidx, lend+1):
                merged[mer_idx] = arr[i]#output 都是放比較小的ele
                mer_idx +=1 

            break
        
        
    #rewrite back to original arr
    #從 左邊arr最一開始, 用剛才排好的merged rest 複寫
    # arry index 與 merged indx 不同
    index = lstart
    for i in range(merged_output_length):
        arr[index] = merged[i]
        index +=1

    return inver_count


#就是 binary serach style
def mergesort_find_invers_with_global_idx(arr,low,hig):
    
    #這裡要不要 =, 是看下面 recursive 參數 low,high 怎麼call
    #下面左右分的 左邊是從 low~mid, 右邊 mid+1~hig, 也就是說
    # mid 這個 這個index 在左邊array 如果base case 沒有 = 的話
    # 是永遠跑不到的, 因此是 low >= hig, 不是 log > hig
    
    #為什麼 bs 只掉 low > hig 即可呢？因為 mid 是被拿來跟target 比較用的
    #mid 拿來比較完就沒用了, 往左分只要給 low~mid-1, 右邊 mid+1~hig
    
    if low >= hig: 
        return 0#0 invers
    
    mid = (low+hig)//2
    
    #left 
    lcount = mergesort_find_invers_with_global_idx(arr,low,mid)
    rcount = mergesort_find_invers_with_global_idx(arr,mid+1,hig)

    low_layer_count = lcount + rcount
    this_layer_count = merge_global_idx(arr,low,mid,mid+1,hig)
    
    
    accumulate_count = low_layer_count + this_layer_count
    return accumulate_count
    
    

In [70]:
def count_inversions(arr):
    # TODO: Complete this function
    #method 1, 2 都 ok
    
    
    #method 1 直接分array 去左右, 不用global index
    mergelist ,rst = mergesort(arr)
    
    #method 2,只是不同input的merge sort
    '''
    low = 0
    hig = len(arr)-1
    rst = mergesort_find_invers_with_global_idx(arr,low,hig)
    # arr 已經被改了
    mergelist = arr
    '''
    print(mergelist)
    return rst
arr = [2, 5, 1, 3, 4]#4
arr = [54, 99, 49, 22, 37, 18, 22, 90, 86, 33]#26
print(count_inversions(arr))


leftarr: [54, 99, 49, 22, 37]
righarr: [18, 22, 90, 86, 33]
leftarr: [54, 99]
righarr: [49, 22, 37]
leftarr: [54]
righarr: [99]
merged: [54, 99]
leftarr: [49]
righarr: [22, 37]
leftarr: [22]
righarr: [37]
merged: [22, 37]
inver_count: 1
inver_count: 2
merged: [22, 37, 49]
inver_count: 2
inver_count: 4
inver_count: 6
merged: [22, 37, 49, 54, 99]
leftarr: [18, 22]
righarr: [90, 86, 33]
leftarr: [18]
righarr: [22]
merged: [18, 22]
leftarr: [90]
righarr: [86, 33]
leftarr: [86]
righarr: [33]
inver_count: 1
merged: [33, 86]
inver_count: 1
inver_count: 2
merged: [33, 86, 90]
merged: [18, 22, 33, 86, 90]
inver_count: 5
inver_count: 9
inver_count: 13
inver_count: 14
inver_count: 15
merged: [18, 22, 22, 33, 37, 49, 54, 86, 90, 99]
[18, 22, 22, 33, 37, 49, 54, 86, 90, 99]
26


<span class="graffiti-highlight graffiti-id_8809fp2-id_8br31oi"><i></i><button>Hide Solution</button></span>

In [ ]:
def count_inversions(arr):
    start_index = 0
    end_index = len(arr) - 1
    output = inversion_count_func(arr, start_index, end_index)
    return output


def inversion_count_func(arr, start_index, end_index):
    if start_index >= end_index:
        return 0

    mid_index = start_index + (end_index - start_index) // 2
    
    # find number of inversions in left-half
    left_answer = inversion_count_func(arr, start_index, mid_index)
    
    # find number of inversions in right-half
    right_answer = inversion_count_func(arr, mid_index + 1, end_index)

    output = left_answer + right_answer
    
    # merge two sorted halves and count inversions while merging
    output += merge_two_sorted_halves(arr, start_index, mid_index, mid_index + 1, end_index)
    return output


def merge_two_sorted_halves(arr, start_one, end_one, start_two, end_two):
    count = 0
    left_index = start_one
    right_index = start_two

    output_length = (end_two - start_two + 1) + (end_one - start_one + 1)
    output_list = [0 for _ in range(output_length)]
    index = 0

    while index < output_length:
        # if left <= right, it's not an inversion
        if arr[left_index] <= arr[right_index]:
            output_list[index] = arr[left_index]
            left_index += 1

        else:
            count = count + (end_one - left_index + 1)           # left > right hence it's an inversion
            output_list[index] = arr[right_index]
            right_index += 1

        index = index + 1

        if left_index > end_one:
            for i in range(right_index, end_two + 1):
                output_list[index] = arr[i]
                index += 1
            break

        elif right_index > end_two:
            for i in range(left_index, end_one + 1):
                output_list[index] = arr[i]
                index += 1
            break

    index = start_one
    for i in range(output_length):
        arr[index] = output_list[i]
        index += 1
    return count

In [68]:
def test_function(test_case):
    arr = test_case[0]
    solution = test_case[1]
    if count_inversions(arr) == solution:
        print("Pass")
    else:
        print("Fail")


In [71]:
arr = [2, 5, 1, 3, 4]
solution = 4
test_case = [arr, solution]
test_function(test_case)

leftarr: [2, 5]
righarr: [1, 3, 4]
leftarr: [2]
righarr: [5]
merged: [2, 5]
leftarr: [1]
righarr: [3, 4]
leftarr: [3]
righarr: [4]
merged: [3, 4]
merged: [1, 3, 4]
inver_count: 2
inver_count: 3
inver_count: 4
merged: [1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
Pass


In [72]:
arr = [54, 99, 49, 22, 37, 18, 22, 90, 86, 33]
solution = 26
test_case = [arr, solution]
test_function(test_case)

leftarr: [54, 99, 49, 22, 37]
righarr: [18, 22, 90, 86, 33]
leftarr: [54, 99]
righarr: [49, 22, 37]
leftarr: [54]
righarr: [99]
merged: [54, 99]
leftarr: [49]
righarr: [22, 37]
leftarr: [22]
righarr: [37]
merged: [22, 37]
inver_count: 1
inver_count: 2
merged: [22, 37, 49]
inver_count: 2
inver_count: 4
inver_count: 6
merged: [22, 37, 49, 54, 99]
leftarr: [18, 22]
righarr: [90, 86, 33]
leftarr: [18]
righarr: [22]
merged: [18, 22]
leftarr: [90]
righarr: [86, 33]
leftarr: [86]
righarr: [33]
inver_count: 1
merged: [33, 86]
inver_count: 1
inver_count: 2
merged: [33, 86, 90]
merged: [18, 22, 33, 86, 90]
inver_count: 5
inver_count: 9
inver_count: 13
inver_count: 14
inver_count: 15
merged: [18, 22, 22, 33, 37, 49, 54, 86, 90, 99]
[18, 22, 22, 33, 37, 49, 54, 86, 90, 99]
Pass


In [73]:
arr = [1, 2, 4, 2, 3, 11, 22, 99, 108, 389]
solution = 2
test_case = [arr, solution]
test_function(test_case)

leftarr: [1, 2, 4, 2, 3]
righarr: [11, 22, 99, 108, 389]
leftarr: [1, 2]
righarr: [4, 2, 3]
leftarr: [1]
righarr: [2]
merged: [1, 2]
leftarr: [4]
righarr: [2, 3]
leftarr: [2]
righarr: [3]
merged: [2, 3]
inver_count: 1
inver_count: 2
merged: [2, 3, 4]
merged: [1, 2, 2, 3, 4]
leftarr: [11, 22]
righarr: [99, 108, 389]
leftarr: [11]
righarr: [22]
merged: [11, 22]
leftarr: [99]
righarr: [108, 389]
leftarr: [108]
righarr: [389]
merged: [108, 389]
merged: [99, 108, 389]
merged: [11, 22, 99, 108, 389]
merged: [1, 2, 2, 3, 4, 11, 22, 99, 108, 389]
[1, 2, 2, 3, 4, 11, 22, 99, 108, 389]
Pass
